In [1]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

/Users/peerasit/senior_project/STELLA-Backend/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare PDF FILE

In [2]:
import pymupdf
from langchain_core.documents import Document

doc = pymupdf.open('../../chunking/pdfs/bts_56-1_2023.pdf')

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1200,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

text_chunks = text_splitter.create_documents([d.get_text() for d in doc])

#### Sematic chunking format markdown

In [3]:
# import pymupdf4llm
# doc2 = pymupdf4llm.to_markdown('../../data_sources/ba_56-1_2023.PDF', page_chunks=True)
# semantic_chunk = SemanticChunker(embeddings=embedding_model, breakpoint_threshold_type="percentile")
# text_chunks = semantic_chunk.create_documents([d["text"] for d in doc2])

#### Semantic Chunking text

In [4]:
# import pymupdf
# from langchain_core.documents import Document

# doc = pymupdf.open("../../chunking/pdfs/bts_56-1_2023.pdf")
# semantic_chunker = SemanticChunker(embeddings=embedding_model, breakpoint_threshold_type="percentile")
# text_chunks = semantic_chunker.create_documents([d.get_text() for d in doc])

#### Hybrid Chunking (Structure Chunking + Semantic Chunking)

In [5]:
# import sys
# sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend/")
# from chunking.one_report_file import oneReportFileChunking

# text_chunks = oneReportFileChunking(content="../../chunking/pdfs/bts_56-1_2023.pdf", file_name="bts_56-1_2023.pdf")

## STELLA CORE DB

In [6]:
import sys
# from pymilvus import db
# sys.path.insert(1, '../../milvus_core/')
sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend")
sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend/milvus")

from milvus.core import Core
from milvus.schema import INDEX_PARAMS, DATA_SOURCE_SCHEMA

# core = Core(schema=DATA_SOURCE_SCHEMA, dense_embedding_model=embedding_model, collection_name="semantic", system_prune=True, database_name="semantic_ragas")
core = Core(
            database_name="semantic_ragas",
            schema=DATA_SOURCE_SCHEMA,
            dense_embedding_model=embedding_model,
            create_first_node=True,
            system_prune_first_node=True,
            token=""
        )

# core.add_document(text_chunks)

[CORE] Initializing Milvus Database Core...
[DB] init Embedding Model...
[DB] init Embedding Model Successfully.
[DB] Found Database: semantic_ragas
[DB] Found Collection "cnode_1".
[DB] Drop Collection "cnode_1"...
cnode_1 has: 46 entities
[DB] Drop Collection "cnode_1" Successfully.
[DB] Create Collection "cnode_1"
[DB] Collection "cnode_1" Is Ready.
[DB] Found Collection "gnode_1".
[DB] Drop Collection "gnode_1"...
gnode_1 has: 0 entities
[DB] Drop Collection "gnode_1" Successfully.
[DB] Create Collection "gnode_1"
[DB] Collection "gnode_1" Is Ready.
[DB] Found Collection "frontend_query_general_documents".
[DB] Drop Collection "frontend_query_general_documents"...
frontend_query_general_documents has: 0 entities
[DB] Drop Collection "frontend_query_general_documents" Successfully.
[DB] Create Collection "frontend_query_general_documents"
[DB] Collection "frontend_query_general_documents" Is Ready.
Create Schma Successfuly.


In [7]:
core.add_document(name="bts", documents=text_chunks, node_type="c", sector_id=20)

<Collection>:
-------------
<name>: cnode_1
<description>: Schema for Data Source Collection
<schema>: {'auto_id': True, 'description': 'Schema for Data Source Collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 8192}}, {'name': 'metadata', 'description': '', 'type': <DataType.JSON: 23>}], 'enable_dynamic_field': True}

<class 'pymilvus.orm.collection.Collection'>
[DB] Commit new Company
[DB] Create New Partition
[DB] Partition bts: 46 entities


In [8]:
# chunk_retriever_vectorStore = core.initVectorStore()
# chunk_retriever = chunk_retriever_vectorStore.as_retriever(search_kwargs={"k" : 1})
config ={
    "k": 1,
    "partition_names": ["bts"],
}
chunk_retriver = core.initVectorStore(collection_name="cnode_1", partition_names=["bts"], search_kwargs=config)

In [9]:
chunk_retriver.invoke("องประกอบของคณะกรรมการบริษัท")

[Document(metadata={}, page_content='การกำกับดูแลกิจการ\nนโยบายการกำกับดูแลกิจการ')]

In [10]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_API_KEY")
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0, max_tokens=4096)

# LLM RAG

In [11]:
from langchain_core.prompts import ChatPromptTemplate

rag_template = """\
Use the following context to answer the user's query. If you cannot answer, please respond with 'I don't know'.

User's Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [12]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0, max_tokens=4096)

recursive_rag_chain = (
    {"context" : chunk_retriver, "question" : RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [13]:
synthetic_data_chunks = text_chunks

# LLM Question

In [14]:
question_prompt = """\
You are a teacher preparing a test. Please create a question that can be answered by referencing the following context.

Context:
{context}
"""

question_prompt = ChatPromptTemplate.from_template(question_prompt)
question_chain = question_prompt | llm | StrOutputParser()

# LLM Ground Truth

In [15]:
ground_truth_prompt = """\
Use the following context and question to answer this question using *only* the provided context.

Question:
{question}

Context:
{context}
"""

ground_truth_prompt = ChatPromptTemplate.from_template(ground_truth_prompt)
ground_truth_chain = ground_truth_prompt | llm | StrOutputParser()

## Create Synthesized Data

In [16]:
questions = []
ground_truths_recursive = []
contexts = []
answers = []

for chunk in synthetic_data_chunks[0:48]:
    # print(chunk)
  questions.append(question_chain.invoke({"context" : chunk.page_content}))
  contexts.append(chunk.page_content)
  ground_truths_recursive.append(ground_truth_chain.invoke({"question" : questions[-1], "context" : str(contexts[-1])}))
  
  answers.append(recursive_rag_chain.invoke(questions[-1]))

# Create Synthesized Data for Evaluation

In [17]:
from datasets import load_dataset, Dataset

qagc_list = []

for question, answer, context, ground_truth in zip(questions, answers, contexts, ground_truths_recursive):
  qagc_list.append({
      "question" : question,
      "answer" : answer,
      "contexts" : [context],
      "ground_truth" : ground_truth
  })

eval_dataset = Dataset.from_list(qagc_list)
eval_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 46
})

In [18]:
eval_dataset[13]


{'question': 'What is the potential risk for shareholders when a company issues additional common shares to support the exercise of rights under warrants, according to the context provided?',
 'answer': 'The potential risk for shareholders when a company issues additional common shares to support the exercise of rights under warrants is control dilution, which may impact the proportion of shares held by investors.',
 'contexts': ['ความเสี่ยงต่อการลงทุนของผู้ถือหลักทรัพย์ (2.2.2)\n • การลงทุนของผู้ถือหลักทรัพย์\nความเสี่ยงจากการออกหุ้นสามัญเพิ่มทุนเพื่อรองรับการใช้สิทธิตามใบสำคัญแสดงของบริษัทฯ อาจส่งผลกระทบต่อ สัดส่วนการถือหุ้นของผู้ลงทุน\n(Control Dilution)\nมาตรการจัดการความเสี่ยง: มี\nหัวข้อความเสี่ยงที่เกี่ยวข้อง\n • การลงทุนของผู้ถือหลักทรัพย์\nความเสี่ยงจากการมีผู้ถือหุ้นรายใหญ่ >25%\nมาตรการจัดการความเสี่ยง: มี\nหัวข้อความเสี่ยงที่เกี่ยวข้อง\n10'],
 'ground_truth': 'The potential risk for shareholders when a company issues additional common shares to support the exercise of right

# Evaluation with RAGAS

In [19]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

result = evaluate(
    eval_dataset,
    metrics=[
        context_precision,
        context_recall,
        # faithfulness,
        # answer_relevancy,
    ],
)
result

Evaluating: 100%|██████████| 92/92 [00:16<00:00,  5.49it/s]


{'context_precision': 0.9565, 'context_recall': 0.8670}

In [20]:
results_df = result.to_pandas()
results_df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall
0,Question: What is the name of the company ment...,[แบบ 56-1 One Report\n(Structured Data Report)...,I don't know.,The name of the company mentioned in the conte...,1.0,1.000000
1,"Based on the context provided, please identify...",[สารบัญ\nหน้า\nการประกอบธุรกิจและผลการดำเนินงา...,I don't know.,Section 1 discusses the structure and operatio...,1.0,1.000000
2,Question: อธิบายความสำคัญของโครงสร้างและการดำเ...,[การประกอบธุรกิจและผลการดำเนินงาน\nโครงสร้างแล...,I don't know.,โครงสร้างและการดำเนินงานของกลุ่มบริษัทเป็นสิ่ง...,1.0,1.000000
3,"Based on the context provided, please provide ...",[นโยบายและภาพรวมการประกอบธุรกิจ (1.1)\nข้อมูลท...,I don't know.,บีทีเอส กรุ๊ป โฮลดิ้งส์ จำกัด (มหาชน),1.0,1.000000
4,"Based on the context provided, please calculat...",[ลักษณะการประกอบธุรกิจ (1.2)\nโครงสร้างรายได้ ...,I don't know.,To calculate the percentage of revenue from op...,1.0,1.000000
5,"Based on the context provided, please calculat...",[รายได้จากในประเทศ (%)\n100.00\n100.00\n100.00...,I don't know.,Total revenue from other sources for the year ...,1.0,1.000000
6,Question: What are two key aspects of business...,[การประกอบธุรกิจและผลการดำเนินงาน\nการบริหารจั...,Two key aspects of business operations and man...,Two key aspects of business operations and man...,1.0,1.000000
7,"Based on the context provided, please explain ...",[การบริหารจัดการความเสี่ยง\nปัจจัยความเสี่ยงต่...,The factors that contribute to strategic risk ...,Factors that contribute to strategic risk for ...,1.0,1.000000
8,"Based on the context provided, please explain ...",[ความเสี่ยงด้านกลยุทธ์และการประกอบธุรกิจ (Stra...,Strategic risk refers to the potential for adv...,Strategic risk in business operations refers t...,1.0,0.250000
9,"Based on the context provided, please explain ...",[• การดำเนินโครงการในอนาคตไม่เป็นไปตามแผน\n • ...,Relying on a small number of suppliers or dist...,Relying on a small number of suppliers or dist...,1.0,0.500000


In [21]:
eval_dataset[35]

{'question': 'Based on the context provided, please answer the following question:\n\nWhat is the position held by นาย ชิ เคือง คง on April 1, 2558?',
 'answer': 'นาย ชิ เคือง คง held the position of Deputy Managing Director on April 1, 2558.',
 'contexts': ['กำกับดูแล\nนาย ชิ เคือง คง\nเพศ: ชาย\nอายุ: 49 ปี\nวุฒิการศึกษา: ปริญญาโท\nสาขา: บริหารธุรกิจ\n3.\nรองกรรมการผู้อำนวยการ\nใหญ่\n1 เม.ย. 2558\nประกันภัยและประกันชีวิต, พัฒนา\nอสังหาริมทรัพย์, สื่อและสิ่งพิมพ์,\nขนส่งและโลจิสติกส์, การเงิน, การ\nตลาดดิจิทัล, ผู้นำ, การจัดการ\nกลยุทธ์, การจัดการความเสี่ยง, การ\nจัดทำงบประมาณ, เทคโนโลยี\nสารสนเทศและการสื่อสาร, อาหาร\nและเครื่องดื่ม\n23'],
 'ground_truth': 'นาย ชิ เคือง คง held the position of Deputy Managing Director on April 1, 2558.'}

In [22]:
results_df['context_precision'].mean()

0.9565217390347824

In [23]:
results_df['context_recall'].mean()

0.8670289855072464